https://dedupeio.github.io/dedupe-examples/docs/csv_example.html

In [1]:
import os

In [2]:
import csv
import re

In [3]:
import logging

In [4]:
import optparse

In [5]:
from future.builtins import next

In [6]:
import dedupe

In [7]:
from unidecode import unidecode

In [8]:
optp = optparse.OptionParser()
optp.add_option('-v', '--verbose', dest='verbose', action='count',
                help='Increase verbosity (specify multiple times for more)'
                )
(opts, args) = optp.parse_args()
log_level = logging.WARNING 
if opts.verbose:
    if opts.verbose == 1:
        log_level = logging.INFO
    elif opts.verbose >= 2:
        log_level = logging.DEBUG
logging.getLogger().setLevel(log_level)

Usage: ipykernel_launcher.py [options]

ipykernel_launcher.py: error: no such option: -f


SystemExit: 2

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
%tb

In [8]:
os.getcwd()

'/home/jovyan'

In [9]:
directory = '/home/jovyan/work/dedupe/dedupe-examples/csv_example/'
input_file = directory + 'csv_example_messy_input.csv'
output_file = directory + 'csv_example_output.csv'
settings_file = directory + 'csv_example_learned_settings'
training_file = directory + 'csv_example_training.json'

In [10]:
def preProcess(column):
    try : # python 2/3 string differences
        column = column.decode('utf8')
    except AttributeError:
        pass
    column = unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    
    if not column:
        column = None
    return column

In [11]:
def readData(filename):
#
    data_d = {}
    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in reader:
            clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
            row_id = int(row['Id'])
            data_d[row_id] = dict(clean_row)

    return data_d

print('importing data ...')

importing data ...


In [12]:
data_d = readData(input_file)

In [13]:
if os.path.exists(settings_file):
    print('reading from', settings_file)
    with open(settings_file, 'rb') as f:
        deduper = dedupe.StaticDedupe(f)
else:
    
    
    
    fields = [
        {'field' : 'Site name', 'type': 'String'},
        {'field' : 'Address', 'type': 'String'},
        {'field' : 'Zip', 'type': 'Exact', 'has missing' : True},
        {'field' : 'Phone', 'type': 'String', 'has missing' : True},
        ]
    deduper = dedupe.Dedupe(fields)
    
    
    deduper.sample(data_d, 15000)
    
    if os.path.exists(training_file):
        print('reading labeled examples from ', training_file)
        with open(training_file, 'rb') as f:
            deduper.readTraining(f)
            
    print('starting active labeling...')

INFO:dedupe.canopy_index:Removing stop word  s


starting active labeling...


In [14]:
dedupe.consoleLabel(deduper)

Site name : henry booth house - bernard gentry
Address : 2326 s dearborn st
Zip : None
Phone : 8428557

Site name : henry booth house - gentry
Address : 2326 s. dearborn
Zip : None
Phone : 8428557

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


y


Site name : hull house association fcch-monica pierce
Address : 1644 s avers st
Zip : 60623
Phone : 2355342

Site name : hull house association fcch-sherisse holmes
Address : 4119 w 16th st
Zip : 60623
Phone : 2355342

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


y


INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(ExistsPredicate: (Exists, Zip), TfidfTextCanopyPredicate: (0.8, Address))
Site name : carole robertson center for learning - carole robertson center ctr 2020
Address : 2020 w roosevelt rd
Zip : None
Phone : 2437300

Site name : easter seals society of metropolitan chicago - gilchrist-marchman
Address : 1001 w roosevelt rd
Zip : None
Phone : 9395115

2/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

Site name : irving park early learning center
Address : 3023 w. montrose
Zip : None
Phone : 5397422

Site name : irving park early learning center
Address : 3023 w montrose avenue
Zip : 60618
Phone : 5397422

2/10 positive, 3/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


y


Site name : lutheran social services of illinois - englewood messiah
Address : 1910 w 64th st
Zip : None
Phone : None

Site name : claremont
Address : 2300 w. 64th st
Zip : None
Phone : 5358110

3/10 positive, 3/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f


Finished labeling
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (alphaNumericPredicate, Phone), SimplePredicate: (commonThreeTokens, Site name))


In [15]:
deduper.train()

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
/opt/conda/lib/python3.6/site-packages/rlr/crossvalidation.py:122: RuntimeWarning: invalid value encountered in double_scalars
  * (true_distinct + false_distinct)))
INFO:rlr.crossvalidation:optimum alpha: 1.000000, score 0.26666666666666666
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (alphaNumericPredicate, Phone), SimplePredicate: (commonThreeTokens, Site name))


In [16]:
with open(training_file, 'w') as tf:
        deduper.writeTraining(tf)

In [17]:
 with open(settings_file, 'wb') as sf:
        deduper.writeSettings(sf)

In [18]:
threshold = deduper.threshold(data_d, recall_weight=1)

INFO:dedupe.api:Maximum expected recall and precision
INFO:dedupe.api:recall: 0.998
INFO:dedupe.api:precision: 0.875
INFO:dedupe.api:With threshold: 0.479


In [19]:
print('clustering...')
clustered_dupes = deduper.match(data_d, threshold)

print('# duplicate sets', len(clustered_dupes))

clustering...
# duplicate sets 517


In [20]:
cluster_membership = {}
cluster_id = 0
for (cluster_id, cluster) in enumerate(clustered_dupes):
    id_set, scores = cluster
    cluster_d = [data_d[c] for c in id_set]
    canonical_rep = dedupe.canonicalize(cluster_d)
    for record_id, score in zip(id_set, scores):
        cluster_membership[record_id] = {
            "cluster id" : cluster_id,
            "canonical representation" : canonical_rep,
            "confidence": score
        }

In [21]:
singleton_id = cluster_id + 1

In [22]:
with open(output_file, 'w') as f_output, open(input_file) as f_input:
    writer = csv.writer(f_output)
    reader = csv.reader(f_input)

    heading_row = next(reader)
    heading_row.insert(0, 'confidence_score')
    heading_row.insert(0, 'Cluster ID')
    canonical_keys = canonical_rep.keys()
    for key in canonical_keys:
        heading_row.append('canonical_' + key)

    writer.writerow(heading_row)

    for row in reader:
        row_id = int(row[0])
        if row_id in cluster_membership:
            cluster_id = cluster_membership[row_id]["cluster id"]
            canonical_rep = cluster_membership[row_id]["canonical representation"]
            row.insert(0, cluster_membership[row_id]['confidence'])
            row.insert(0, cluster_id)
            for key in canonical_keys:
                row.append(canonical_rep[key].encode('utf8'))
        else:
            row.insert(0, None)
            row.insert(0, singleton_id)
            singleton_id += 1
            for key in canonical_keys:
                row.append(None)
        writer.writerow(row)

In [23]:
import pandas as pd

In [24]:
dfOutput = pd.read_csv(output_file)
dfOutput.head(20)

,Cluster ID,confidence_score,Id,Source,Site name,Address,Zip,Phone,Fax,Program Name,...,canonical_Executive Director,canonical_Center Director,canonical_ECE Available Programs,canonical_NAEYC Valid Until,canonical_NAEYC Program Id,canonical_Email Address,canonical_Ounce of Prevention Description,canonical_Purple binder service type,canonical_Column,canonical_Column2
0,219,0.922147,0,CPS_Early_Childhood_Portal_scrape.csv,Salvation Army - Temple / Salvation Army,1 N Ogden Ave,NaN,2262649.0,NaN,Child Care,...,b'',b'',b'cp',b'',b'',b'',b'',b'',b'',b''
1,219,0.922147,1,CPS_Early_Childhood_Portal_scrape.csv,Salvation Army - Temple / Salvation Army,1 N Ogden Ave,NaN,2262649.0,NaN,Child Care,...,b'',b'',b'cp',b'',b'',b'',b'',b'',b'',b''
2,220,0.836727,2,CPS_Early_Childhood_Portal_scrape.csv,National Louis University - Dr. Effie O. Elli...,10 S Kedzie Ave,NaN,5339011.0,NaN,Child Care,...,b'4364ps(hs collaboration with childcare & prek)',b'www.ymcachicago.org',b'dorothy cole-gary',b'anelyn jablo',b'',b'',b'',b'',b'',b''
3,220,0.836727,3,CPS_Early_Childhood_Portal_scrape.csv,National Louis University - Dr. Effie O. Elli...,10 S Kedzie Ave,NaN,5339011.0,NaN,Child Care,...,b'4364ps(hs collaboration with childcare & prek)',b'www.ymcachicago.org',b'dorothy cole-gary',b'anelyn jablo',b'',b'',b'',b'',b'',b''
4,221,0.946079,4,CPS_Early_Childhood_Portal_scrape.csv,Board Trustees-City Colleges of Chicago - Oli...,10001 S Woodlawn Ave,NaN,2916100.0,NaN,Child Care,...,b'',b'',b'',b'',b'',b'',b'',b'',b'',b''
5,221,0.946079,5,CPS_Early_Childhood_Portal_scrape.csv,Board Trustees-City Colleges of Chicago - Oli...,10001 S Woodlawn Ave,NaN,2916100.0,NaN,Child Care,...,b'',b'',b'',b'',b'',b'',b'',b'',b'',b''
6,222,0.946079,6,CPS_Early_Childhood_Portal_scrape.csv,Easter Seals Society of Metropolitan Chicago ...,1001 W Roosevelt Rd,NaN,9395115.0,NaN,Child Care,...,b'',b'',b'',b'',b'',b'',b'',b'',b'',b''
7,222,0.946079,7,CPS_Early_Childhood_Portal_scrape.csv,Easter Seals Society of Metropolitan Chicago ...,1001 W Roosevelt Rd,NaN,9395115.0,NaN,Child Care,...,b'',b'',b'',b'',b'',b'',b'',b'',b'',b''
8,223,0.926939,8,CPS_Early_Childhood_Portal_scrape.csv,Hull House Association - Uptown Head Start / ...,1020 W Bryn Mawr Ave,NaN,7695753.0,NaN,Child Care,...,b'7027ps(hs collaboration with childcare & prek)',b'',b'',b'',b'',b'',b'',b'',b'',b''
9,224,0.878407,9,CPS_Early_Childhood_Portal_scrape.csv,Hull House Association - Child Dev. Central O...,1030 W Van Buren St,NaN,9068600.0,NaN,Child Care,...,b'',b'',b'',b'',b'',b'',b'',b'',b'',b'child care'
